# Data Preprocessing

In [1]:
import pandas as pd

C:\Users\USER\AppData\Local\Temp\ipykernel_22272\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# german_file_path="Data_Sets/deu.txt"
# spanish_file_path="Data_Sets/data.csv"

In [3]:
# Initialize lists to store German and English data
# german_data = []
# english_data = []

# with open(german_file_path,'r',encoding='utf-8') as file:
#     for line in file:
#         english,german,_=line.strip().split('\t')
#         english_data.append(english)
#         german_data.append(german)
        

# # Create a DataFrame using Pandas
# df = pd.DataFrame({ 'English': english_data , 'German': german_data})

# # Display the first few rows of the DataFrame
# print(df[:20])        

   English                    German
0      Go.                      Geh.
1      Hi.                    Hallo!
2      Hi.                Grüß Gott!
3     Run!                     Lauf!
4     Run.                     Lauf!
5     Wow!               Potzdonner!
6     Wow!             Donnerwetter!
7    Fire!                    Feuer!
8    Help!                    Hilfe!
9    Help!                  Zu Hülf!
10   Stop!                    Stopp!
11   Wait!                    Warte!
12   Wait.                    Warte.
13  Begin.                  Fang an.
14  Go on.              Mach weiter.
15  Hello!                    Hallo!
16  Hurry!               Beeil dich!
17  Hurry!                  Schnell!
18  I hid.      Ich versteckte mich.
19  I hid.  Ich habe mich versteckt.


In [4]:
# df_spanish=pd.read_csv(spanish_file_path)

# # renaming the names
# new_column_names={'english':'English','spanish':'Spanish'}
# df_spanish=df_spanish.rename(columns=new_column_names)

# print(df_spanish[:10])

  English     Spanish
0     Go.         Ve.
1     Go.       Vete.
2     Go.       Vaya.
3     Go.     Váyase.
4     Hi.       Hola.
5    Run!     ¡Corre!
6    Run.     Corred.
7    Who?     ¿Quién?
8   Fire!     ¡Fuego!
9   Fire!  ¡Incendio!


In [6]:
# file_name=f"data_german.csv"

# df.to_csv(f"Data_Sets/{file_name}",index=False)
# print("DataFrame saved successfully as", file_name)

DataFrame saved successfully as data_german.csv


In [7]:
import os
# current_file_path="Data_Sets/data.csv"
# new_file_path="Data_Sets/data_spanish.csv"

# os.rename(current_file_path,new_file_path)

# print("File renamed successfully to", new_file_path)

File renamed successfully to Data_Sets/data_spanish.csv


In [4]:
import pandas as pd
german_file_path="Data_Sets/data_german.csv"
spanish_file_path="Data_Sets/data_spanish.csv"

In [5]:
df1=pd.read_csv(german_file_path)
df2=pd.read_csv(spanish_file_path)

In [10]:
import random
import tensorflow as tf
import string
import re
import csv
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
df1.head(20)

,English,German
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!
3,Run!,Lauf!
4,Run.,Lauf!
5,Wow!,Potzdonner!
6,Wow!,Donnerwetter!
7,Fire!,Feuer!
8,Help!,Hilfe!
9,Help!,Zu Hülf!


In [18]:
first_column_values_last_10_rows=df1['English'].tail(10)


paragraph = '\n'.join(first_column_values_last_10_rows)
# Print the paragraph
print(paragraph)

As a prank, some students let three goats loose inside their school after painting the numbers 1, 2 and 4 on the sides of the goats. The teachers spent most of the day looking for goat number 3.
The small crowd at Hiroshima Peace Memorial Park stood for a moment of silence at 8:15 a.m., the exact moment an atomic bomb nicknamed “Little Boy” was dropped from the U.S. warplane Enola Gay.
In today's world, we have to equip all our kids with an education that prepares them for success, regardless of what they look like, or how much their parents make, or the zip code that they live in.
Death is something that we're often discouraged to talk about or even think about, but I've realized that preparing for death is one of the most empowering things you can do. Thinking about death clarifies your life.
At a moment when our economy is growing, our businesses are creating jobs at the fastest pace since the 1990s, and wages are starting to rise again, we have to make some choices about the kind o

In [19]:
len(df1)

221533

# Split the English and german translation pairs

In [26]:
#common function for both spanish and german

class split_pairs:
 def split_pairs_method(self,df1): 
  text_pairs=[]
  i=0
  for i in range(len(df1)):
    english,language=df1["English"][i],df1["German"][i]
    german="[start] "+language+" [end]"
    text_pairs.append((english,language))

  return text_pairs    


In [27]:
#randomly selecting that if the above function work
class random_pair_test:
  def random_test_method(self,text_pairs): 
   for i in range(3):
    print(random.choice(text_pairs))
    

In [28]:
#pairing for german text

german_text_pairs=split_pairs().split_pairs_method(df1)

random_pair_test().random_test_method(german_text_pairs)

('Your guess is as good as mine.', 'Deine Vermutung ist so gut wie meine.')
('Try on both these coats and compare them.', 'Probieren Sie beide Mäntel an und vergleichen Sie sie.')
("I can't give you everything you want.", 'Ich kann dir nicht alles geben, was du willst.')


# Randomizing the data

In [29]:
random.shuffle(german_text_pairs)

# Split the data into training, validation,testing

In [32]:
#class for splitting text pairs in to train,test,validation
class splitting:
    def splitting_method(self,text_pairs):
        num_val_sample=int(0.15*len(text_pairs))
        num_train_samples=len(text_pairs) - 2 * num_val_sample
        train_pairs=text_pairs[:num_train_samples]
        val_pairs=text_pairs[num_train_samples:num_train_samples+num_val_sample]
        test_pairs=text_pairs[num_train_samples+num_val_sample:]
        
        print("Total Sentences: ",len(text_pairs))
        print("Training set size: ",len(train_pairs))
        print("Validation set size: ",len(val_pairs))
        print("Testng set size: ",len(test_pairs))
        return train_pairs,val_pairs,test_pairs    



In [33]:
german_train_pairs,german_val_pairs,german_test_pairs=splitting().splitting_method(german_text_pairs)


Total Sentences:  221533
Training set size:  155075
Validation set size:  33229
Testng set size:  33229


In [35]:
print(len(german_test_pairs)+len(german_train_pairs)+len(german_val_pairs))

221533


# Removing Puncuations